# Skill estimation using Stan

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(66)
import pickle

## Model that defined skill's distribution with more features

In [2]:
skill_model = """
data {
  int<lower=1> N;             // Total number of players
  int<lower=1> E;             // number of games
  real<lower=0> scale;        // scale value for probability computation
  int<lower=0,upper=1> win[E]; // PA wins vs PB
  int PA[E];                  // player info between each game
  int PB[E];                  // 
  int SA[E];              // scores for each game
  int SB[E];
}
parameters {
  vector [N] skill;           // skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
    if (skill[PA[i]]*0.3 + 1 > 0){SA[i] ~ poisson( skill[PA[i]]*0.3 + 1 );}
    if (skill[PB[i]]*0.3 + 1 > 0){SB[i] ~ poisson( skill[PB[i]]*0.3 + 1 );}
  }   // win probability is a logit function of skill difference
}
"""

    if (skill[PA[i]]*0.3 + 1 > 0){SA[i] ~ poisson( skill[PA[i]]*0.3 + 1 );}
    if (skill[PB[i]]*0.3 + 1 > 0){SB[i] ~ poisson( skill[PB[i]]*0.3 + 1 );}
    
    
    SA[i] ~ poisson( fabs(skill[PA[i]]*0.3 + 1) );
    SB[i] ~ poisson( fabs(skill[PB[i]]*0.3 + 1) );    

Now, compile the model.  

In [3]:
sm = pystan.StanModel(model_code = skill_model)

In [4]:
with open('skill_model_scores.pkl', 'wb') as f: pickle.dump(sm, f)

In [85]:
# try:     # load it if already compiled
#     sm = pickle.load(open('skill_model_scores.pkl', 'rb'))
# except:  # ow, compile and save compiled model
#     sm = pystan.StanModel(model_code = skill_model)
#     with open('skill_model_scores.pkl', 'wb') as f: pickle.dump(sm, f)

## Processing data

In [22]:
def load_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins = []
    playerA, playerB = [], []
    raceA, raceB = [], []
    nplayers = len(playerid)
    nplays = np.zeros( (nplayers,nplayers) )
    race = {'P':0, 'T':1, 'Z':2}
    scoresA = []
    scoresB = []
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                
                playerA.append(a+1)
                playerB.append(b+1)
                wins.append(1 if aw else 0) 
                raceA.append(race[csv[6]])
                raceB.append(race[csv[7]])
                sa, sb = csv[3].split('–')
#                 scores.append(abs(int(sa)-int(sb)))
#                 scores.append((int(sa)-int(sb)))
                scoresA.append(int(sa))
                scoresB.append(int(sb))

    return playerid,nplayers,playerA,playerB,raceA, raceB, wins, scoresA, scoresB

In [23]:
playerid,nplayers,playerA,playerB,raceA, raceB, wins, scoresA, scoresB = load_data()

In [7]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])
print('raceA', raceA[:10])
print('raceB', raceB[:10])
print('scoresA', scoresA[:10])
print('scoresB', scoresB[:10])

summary: 
# players 999
# games 4678
player A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
player B [2, 3, 4, 5, 6, 7, 8, 9, 10, 9]
wins [0, 0, 0, 0, 1, 1, 1, 0, 0, 1]
raceA [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
raceB [0, 2, 1, 2, 0, 1, 2, 0, 1, 0]
scoresA [0, 1, 0, 0, 1, 1, 1, 0, 0, 2]
scoresB [2, 2, 2, 1, 0, 0, 0, 2, 2, 1]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [8]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.5,
    'win':wins,
    'PA': playerA,
    'PB': playerB,
    'SA': scoresA,
    'SB': scoresB
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [9]:
fit = sm.sampling(data=skill_data, iter=2000, chains=4, n_jobs = 4, verbose = True)

In [10]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [11]:
samples

OrderedDict([('skill',
              array([[ 1.67208284,  3.11666643,  3.73705795, ..., -8.10203255,
                      -0.86255959, -0.45296302],
                     [ 1.62864557,  2.79543307,  4.0622911 , ..., -8.11538916,
                      -0.20072909, -0.27095842],
                     [ 1.54899386,  2.5547274 ,  3.73025306, ..., -7.9237259 ,
                      -0.08645157, -0.30769752],
                     ...,
                     [ 1.7508092 ,  0.96687758,  2.06992011, ..., -3.89341279,
                      -1.07046627, -2.46226095],
                     [ 1.75120358,  1.02881859,  2.3823339 , ..., -3.66886147,
                      -1.1556531 , -2.41340684],
                     [ 1.74052672,  0.97821856,  2.08125552, ..., -3.54860807,
                      -1.32271144, -2.36110503]])),
             ('lp__',
              array([-9509.43735666, -9513.39258552, -9494.015467  , ...,
                     -9490.09318035, -9489.99649636, -9490.57315838]))])

In [12]:
samples['skill'].shape # 2*100 iterations? 999 players

(4000, 999)

Remember to save the prediction!

In [13]:
with open('skill_hat_scores.pkl', 'wb') as f: 
    pickle.dump(samples, f)

## Sample Model Evaluation

In [14]:
skill_hat = pickle.load(open('skill_hat_scores.pkl', 'rb'))

In [15]:
skill_hat = skill_hat['skill']

In [42]:
def load_valid_data(playerid,dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')


  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    
    games = []
    race = {'P':0, 'T':1, 'Z':2}
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10 or csv[6] == 'R' or csv[7] == 'R':
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                nwins[a,b] += aw
                nwins[b,a] += bw
                
                sa, sb = csv[3].split('–')
#                 scores.append(abs(int(sa)-int(sb)))
#                 scores.append((int(sa)-int(sb)))
                scoresA.append(int(sa))
                scoresB.append(int(sb))
                games.append((a,b,aw,sa,sb))
    
    
    return nplayers, nplays, nwins, games

In [43]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data(playerid)

In [44]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

summary:  999
(999, 999) 9542.0
(999, 999) 4788.0
games 4771


In [35]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = logit( skill_data['scale']*(skill[:,i]-skill[:,j]) ).mean()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
            

    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [36]:
loss = prediction_loss(skill_hat, nplayers_val, nplays_val, nwins_val, games_val)

In [37]:
loss

(0.26739516078822506, 0.18093319906008729)

In [40]:
def loss_per_game(skill, games):
    
    loss = 0.
    
    for game in games:
        a, b, w, sa, sb = game
        prob_hat = logit( skill_data['scale']*(skill[:,a]-skill[:,b]) ).mean()
        loss += np.logical_xor(prob_hat >= 0.5, w)
        
    loss /= len(games)*2
    
    return loss


In [41]:
loss_per_game(skill_hat, games_val)

0.21798365122615804